<a href="https://colab.research.google.com/github/CesarPortales/BootCamp/blob/master/DATA_LAKES.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# DATA LAKES

![texto alternativo](https://knowledgent.com/wp-content/uploads/2014/09/The-Supervised-Data-Lake_GraphicOnly.png)

## Características

* Uso de múltiples herramientas y productos
* Especificación del dominio
* Gestión automatizada de metadatos.
* Flujos de trabajo de ingestión configurables.

## Metadatos???

- Datos acerca de los datos (cantidad, dominio, naturaleza, extractos, etc)
- Governance
- La preparación y limpieza de datos requieren un conocimiento sólido de qué conjunto de datos está trabajando. 
- Diferentes conjuntos requieren diferentes tipos de preparación, en función de lo que has aprendido sobre ellos históricamente.
- Limpiar datos y prevenir errores.
- Las licencias pueden requerir restricciones de acceso y otro tratamiento especial de algunas fuentes de datos. (GDPR, etc)

## Diseño de un Data lake

### Definición la estrategia del DAaaS

Definir el catálogo de servicios que proporcionará la plataforma DAaaS, que incluye incorporación de datos, limpieza de datos, transformación de datos, datapedias, bibliotecas de herramientas analíticas y otros.


### Arquitectura DAaaS

Definir la selección de componentes, la definición de procesos de ingeniería y el diseño de interfaces de usuario.
Diseño y ejecución de Proofs-of-Concept (PoC) para demostrar la viabilidad del enfoque DAaaS.


### DAaaS Operating Model Design and Rollout

Personalizar los modelos operativos DAaaS para cumplir con los procesos, la estructura organizacional, las reglas y el gobierno de los clientes individuales. 
Realizar seguimiento de consumo y mecanismos de informe.


### Desarrollo de la plataforma DAaaS.

Construcción iterativa de todas las capacidades de la plataforma, incluido el diseño, desarrollo e integración, **pruebas**, carga de datos, metadatos y población de catálogos, y despliegue.


## Retos de un Data Lake

### Timeliness
El acceso a datos debe ser  rapido
los usuarios gastaran tiempo en sacar datos de sql db, guardarla y evaluarla por su cuenta: NO!


### Flexibility

acceso a datos con herramientas propias de cada usuario
el usuario debe ser capaz de analizar datos de cualquier formato


### Quality

el usuario no debe dudar de los datos
trust is everything, quality is everything
si no confian, haran circumvent <- FATAL!


### Findability. 

Al haber muchos datos necesitas que los usuarios puedan encontrar datos rapido

![texto alternativo](https://i.imgur.com/GrVszRZ.png)

## Mas lectura

- https://tdwi.org/articles/2017/10/16/arch-all-data-lake-manifesto-10-best-practices.aspx

# VOLVAMOS A HADOOP!!!

Como habiamos hablado, hadoop son 4 elementos principales:

- HDFS
- MapReduce
- YARN
- Common

## Venga, vamos con YARN

Que podemos hacer con YARN?

1.    Primero siempre entramos en el directorio de instalacion de hadoop de la VM:
```
cd $HADOOP_PREFIX
```
2.   Luego podemos ejecutar jobs de mapreduce con `bin/yarn` o `bin/hadoop` :

```
bin/yarn jar share/hadoop/mapreduce/hadoop-mapreduce-examples-2.7.1.jar pi 10 10

bin/hadoop jar share/hadoop/mapreduce/hadoop-mapreduce-examples-2.7.1.jar pi 10 10
```

La diferencia radica en como *YARN* va a manejar los recursos internamente y podeis leer al respecto aqui: 
https://stackoverflow.com/questions/22769129/differences-between-hadoop-jar-and-yarn-jar


Para ver lista de jobs en ejecucion

```
./bin/hadoop job -list
```

Para matar un job:

```
./bin/yarn application -kill XXXAPPIDXXX
```



### Nueva URL para jugar!

http://127.0.0.1:8088/cluster

Podemos:
- Ver historial
- Matar applications
- Ver detalles de aplicaciones

### Logs

Todos los logs: 
- http://127.0.0.1:8088/logs/
- http://127.0.0.1:8088/logs/userlogs/

### Scheduling

Se utilizan colas, hay una cola principal llamada root, el resto se pueden definir de la siguiente forma:

![job scheduling](http://blog.cloudera.com/wp-content/uploads/2015/12/untangling-yarn-3-f1.png)

```
./bin/yarn jar share/hadoop/mapreduce/hadoop-mapreduce-examples-2.7.1.jar pi 2 3 &
./bin/yarn jar share/hadoop/mapreduce/hadoop-mapreduce-examples-2.7.1.jar pi 2 3 &
./bin/yarn jar share/hadoop/mapreduce/hadoop-mapreduce-examples-2.7.1.jar pi 2 3 &
```


Otra URL! http://127.0.0.1:8088/cluster/scheduler

***NOTA***: La personalización del Fair Job Scheduler generalmente implica la alteración de dos archivos. 


En primer lugar, las opciones de todo el planificador se pueden establecer agregando propiedades de configuración en el archivo yarn-site.xml en su directorio de configuración existente. 

1.    Primero entramos en el directorio donde se encuentran los ficheros de configuracion:
```
cd /usr/local/hadoop/etc/hadoop
```
2.    Con esto podemos ver los contenidos del yarn-site.xml (El fichero de configuracion principal de YARN)
```
cat yarn-site.xml
```
3.   Para editarlo debemos utilizar vi (https://txikano.files.wordpress.com/2006/12/manual_rapido_vi.pdf):
```
vi yarn-site.xml
```

Copiar esto dentro:

```
<property>
  <name>yarn.resourcemanager.scheduler.class</name>
  <value>org.apache.hadoop.yarn.server.resourcemanager.scheduler.fair.FairScheduler</value>
</property>
<property>
  <name>yarn.scheduler.fair.allocation.file</name>
  <value>/usr/local/hadoop/etc/hadoop/fair-scheduler.xml</value>
</property>
```

Visitar http://127.0.0.1:8088/conf y buscar scheduler.

Segundo, en la mayoría de los casos, los usuarios querrán crear un archivo de asignación que enumere qué colas existen y sus respectivos pesos y capacidades. El archivo de asignación se vuelve a cargar cada 10 segundos, lo que permite realizar cambios sobre la marcha.

```
vi fair-scheduler.xml
```

Copiar esto dentro: 
NOTA: Esto se puede configurar mucho mas profudamente. Poner colas padre, etc.

```
<?xml version="1.0"?>
<allocations>
  <queue name="test_queue_80">
    <weight>80.0</weight>
  </queue>
  <queue name="test_queue_20">
  <weight>20.0</weight>
  </queue>
</allocations>
```


Para retomar estos cambios hay que reiniciar YARN en todos los nodos:

```
cd $HADOOP_PREFIX
./sbin/stop-yarn.sh
./sbin/start-yarn.sh
```

Visitar http://127.0.0.1:8088/conf

Documentacion de scheduler: https://hadoop.apache.org/docs/r2.7.4/hadoop-yarn/hadoop-yarn-site/FairScheduler.html


## Volvamos a la teoria un momento

YARN al final no es mas que una capa de control sobre las aplicaciones o comandos que corren sobre hadoop.

Es una capa muy cercana al HDFS y de bastante bajo nivel, hecha en JAVA.

![yarn arch](https://www.oreilly.com/library/view/hadoop-the-definitive/9781491901687/images/hddg_0401.png)

### Que sucede cuando YARN supervisa una ejecucion?

![yarn process](https://www.oreilly.com/library/view/hadoop-the-definitive/9781491901687/images/hddg_0402.png)

### Mas lectura
- https://www.oreilly.com/library/view/hadoop-the-definitive/9781491901687/ch04.html


## De vuelta a la instancia

### Como enviar un job a una cola especifica?

```
./bin/yarn jar share/hadoop/mapreduce/hadoop-mapreduce-examples-2.7.1.jar pi -D mapred.job.queue.name=root.test_queue_20 -D mapred.job.name=test20 100 200 &
./bin/yarn jar share/hadoop/mapreduce/hadoop-mapreduce-examples-2.7.1.jar pi -D mapred.job.queue.name=root.test_queue_80 -D mapred.job.name=test80 100 120 &

bin/mapred job -logs JOBID

```

Ahora visitemos:
http://127.0.0.1:8088/cluster/scheduler

Puede ser tan complejo como:

![complex yarn](http://blog.cloudera.com/wp-content/uploads/2016/01/untangling-yarn-3-f2.png)

# Conclusion

Con la arquitectura provista por HDFS + YARN + MAP REDUCE podemos obtener un modelo de arquitectura big data inicial, faltan muchas herramientas y conceptos como:

- Seguridad
- Gestion de Usuarios
- Deployment
- Testing
- CI/CD
- Cluster design
- SEGURIDAD!
- Cloud vs in premise
- Flujo de trabajo


#### Un entorno de hadoop + yarn apunta hacia este diseño:

![yarn total](https://2xbbhjxc6wk3v21p62t8n4d4-wpengine.netdna-ssl.com/wp-content/uploads/2014/07/data.png)


#### Y aun asi!

Encima de esto pueden ir mas cosas, o al lado, etc (Spark, KAFKA, etc)... Pero ya veremos mas cosas.


## BONUS COMIC:

![texto alternativo](https://marketoonist.com/wp-content/uploads/2014/01/140113.bigdata.jpg)

